## Лабораторная работа №6
### Задание 1
Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.
Если не установлен модуль CatBoost необходимо в консоль ввести команду pip install catboost

### Выполнение работы:
#### Шаг 1. Загрузка датасета

In [1]:
!pip install catboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm


df = pd.read_csv("./base_v3.csv")

y = df["is_fraud"]
X = df.drop("is_fraud", axis=1)


# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


#### Шаг 3. Метод CatBoost
CatBoost - это высокоэффективная библиотека градиентного бустинга, разработанная для эффективной работы с категориальными признаками. Она автоматически обрабатывает категориальные признаки, не требуя их предварительного кодирования. Вот некоторые гиперпараметры CatBoost и их описание:

### depth (глубина дерева):

Описание: Максимальная глубина каждого дерева в ансамбле.
Возможные значения: Любое целое положительное число.

### learning_rate (скорость обучения):

Описание: Коэффициент, определяющий величину шага градиентного спуска. Низкая скорость обучения может улучшить обобщающую способность модели.
Возможные значения: Любое положительное число.

### l2_leaf_reg (L2 регуляризация):

Описание: Сила L2-регуляризации для листьев дерева.
Возможные значения: Любое положительное число.

### iterations (количество итераций):

Описание: Количество деревьев в ансамбле (аналогично n_estimators в других моделях).
Возможные значения: Любое целое положительное число.

### loss_function (функция потерь):

Описание: Функция потерь, которая оптимизируется во время обучения.
Возможные значения: 'MultiClass' (для многоклассовой классификации) или 'Logloss' (логистическая регрессия).

In [2]:
# Определение параметров для CatBoost
param_grid_catboost = {
    'depth': [1, 4, 6, 7, 10],
    'learning_rate': [0.01, 0.1, 1],
    'l2_leaf_reg': [1, 3, 5, 9],
    'iterations': [100, 200,],
    'loss_function': ['MultiClass', 'Logloss']
}

# Создание модели CatBoost
catboost = CatBoostClassifier()

# Подбор оптимальных параметров с помощью перекрестной проверки для CatBoost
grid_search_catboost = GridSearchCV(catboost, param_grid_catboost, cv=5,n_jobs=-1)
grid_search_catboost.fit(X_train, y_train)

# Получение лучших параметров для CatBoost
best_params_catboost = grid_search_catboost.best_params_
best_score_catboost = grid_search_catboost.best_score_

0:	learn: 0.5748440	total: 154ms	remaining: 30.6s
1:	learn: 0.4920198	total: 158ms	remaining: 15.6s
2:	learn: 0.4276877	total: 162ms	remaining: 10.6s
3:	learn: 0.3710101	total: 166ms	remaining: 8.13s
4:	learn: 0.3258045	total: 170ms	remaining: 6.63s
5:	learn: 0.2923567	total: 174ms	remaining: 5.62s
6:	learn: 0.2510534	total: 178ms	remaining: 4.92s
7:	learn: 0.2218920	total: 183ms	remaining: 4.38s
8:	learn: 0.2031220	total: 187ms	remaining: 3.96s
9:	learn: 0.1947059	total: 191ms	remaining: 3.62s
10:	learn: 0.1872925	total: 195ms	remaining: 3.35s
11:	learn: 0.1733763	total: 200ms	remaining: 3.13s
12:	learn: 0.1634564	total: 204ms	remaining: 2.94s
13:	learn: 0.1573562	total: 209ms	remaining: 2.77s
14:	learn: 0.1511246	total: 213ms	remaining: 2.63s
15:	learn: 0.1466068	total: 218ms	remaining: 2.5s
16:	learn: 0.1431905	total: 222ms	remaining: 2.39s
17:	learn: 0.1399783	total: 227ms	remaining: 2.29s
18:	learn: 0.1373160	total: 231ms	remaining: 2.2s
19:	learn: 0.1330010	total: 236ms	remaining

## Вывод результатов


In [3]:
test_score = grid_search_catboost.score(X_test, y_test)
print("Лучшие параметры для CatBoost:", best_params_catboost)
print("Лучший результат (точность) для CatBoost:", best_score_catboost)
print("Точность на тестовом наборе данных:", test_score)

best_model = grid_search_catboost.best_estimator_
predictions = best_model.predict(X_test)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для CatBoost: {'depth': 6, 'iterations': 200, 'l2_leaf_reg': 3, 'learning_rate': 0.1, 'loss_function': 'Logloss'}
Лучший результат (точность) для CatBoost: 0.9697916666666666
Точность на тестовом наборе данных: 0.9779166666666667

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      2038
           1       0.95      0.90      0.93       362

    accuracy                           0.98      2400
   macro avg       0.97      0.95      0.96      2400
weighted avg       0.98      0.98      0.98      2400

